In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [28]:
questions = pd.DataFrame.from_csv('question_simple.csv', index_col=None)
questions.head()

,QuestionId,QuestionCreation,QuestionLastActivity,AcceptedAnsId,AcceptedDate,QScore,QVotes
0,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,12,12
1,2,2011-11-15T20:50:11.270,2011-12-02T11:38:48.620,10,2011-11-16T00:00:00.000,12,12
2,3,2011-11-15T20:52:27.287,2015-07-18T09:49:54.953,109,2013-01-19T00:00:00.000,26,30
3,7,2011-11-15T20:56:40.383,2015-11-23T12:25:29.887,2977,2012-10-22T00:00:00.000,23,26
4,14,2011-11-15T21:01:05.117,2015-08-13T13:14:56.870,20,2011-11-16T00:00:00.000,11,12


In [29]:
a_questions = pd.DataFrame.from_csv('question_votes.csv', index_col=None)
a_questions[a_questions.AcceptedAnsId.notnull()].head()

,QuestionId,QuestionCreation,QuestionLastActivity,AcceptedAnsId,AcceptedDate,VoteType,QVoteCreation
0,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037
1,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037
2,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037
3,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037
4,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037


In [30]:
get_votes_qv = lambda df: pd.Series((df.VoteType==2).cumsum() + (df.VoteType==3).cumsum(),name='QVotes')
get_score_qv = lambda df: pd.Series((df.VoteType==2).cumsum() - (df.VoteType==3).cumsum(),name='QScore')

predictors_qvotes = ['QuestionId','QuestionCreation','QuestionLastActivity','AcceptedAnsId','AcceptedDate','QVoteCreation']
f_q = lambda df: pd.concat([df[cname] for cname in df.columns.values.tolist() if cname in predictors_qvotes]+[get_score_qv(df),get_votes_qv(df)],axis=1)
a_questions = a_questions.sort_values(by='QVoteCreation').groupby(['QuestionId']).apply(f_q)
a_questions.head(10)

,QuestionId,QuestionCreation,QuestionLastActivity,AcceptedAnsId,AcceptedDate,QVoteCreation,QScore,QVotes
0,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,1,1
11,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,2,2
10,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,3,3
9,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,4,4
7,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,5,5
6,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,6,6
8,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,7,7
4,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,8,8
3,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,9,9
2,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,10,10


In [31]:
a_votes = pd.DataFrame.from_csv('votes-answers.csv', index_col=None)
a_votes = pd.merge(a_votes, a_questions, how='inner', on=['QuestionId'],suffixes=['_v', '_q'])
a_votes.head()

,VoteId,VoteCreation,AnsCreation,VoteType,AnsId,QuestionId,AnsWordCount,QuestionCreation,QuestionLastActivity,AcceptedAnsId,AcceptedDate,QVoteCreation,QScore,QVotes
0,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,1,1
1,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,2,2
2,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,3,3
3,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,4,4
4,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,5,5


In [32]:
predictors_raw_votans =['VoteId','VoteCreation','AnsCreation','VoteType','AnsId','QuestionId','AnsWordCount','QuestionCreation','AcceptedAnsId','AcceptedDate']

valid_qavotes = lambda df: df[df.VoteCreation>=df.QVoteCreation]
#Use twice valid_qavotes, could use once to improve efficiency, but check correctness of index selection
get_max_qv = lambda df: valid_qavotes(df).loc[valid_qavotes(df).QVotes.idxmax(),['QScore','QVotes']].squeeze()
get_latest_qv = lambda df : pd.Series([0,0],index=['QScore','QVotes']) if not (df.VoteCreation>=df.QVoteCreation).any() else get_max_qv(df)
get_head = lambda df: [df[cname].iloc[0] for cname in df.columns.values.tolist() if cname in predictors_raw_votans]
get_qv = lambda df : pd.Series(get_head(df),index=predictors_raw_votans).append(get_latest_qv(df)).to_frame()

a_votes = a_votes.sort_values(by='VoteCreation').groupby(['VoteId']).apply(get_qv).unstack(level=-1).reset_index(level=[0],drop=True)
a_votes.drop(a_votes.columns[[0]], axis=1, inplace=True)
a_votes.columns = a_votes.columns.droplevel()
a_votes.head()

,VoteCreation,AnsCreation,VoteType,AnsId,QuestionId,AnsWordCount,QuestionCreation,AcceptedAnsId,AcceptedDate,QScore,QVotes
0,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0
1,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0
2,2011-11-15T00:00:00.000,2011-11-15T20:54:09.083,2,5,1,37,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0
3,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,3,4,1,190,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0
4,2011-11-15T00:00:00.000,2011-11-15T20:54:49.410,2,6,2,51,2011-11-15T20:50:11.270,10,2011-11-16T00:00:00.000,0,0


In [33]:
a_votes['AcceptedDate'].fillna(pd.to_datetime('20160320'),inplace=True) #Date After Data Set Collection
a_votes['AcceptedAge'] = (pd.to_datetime(a_votes.AcceptedDate,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(a_votes.QuestionCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
a_votes['AcceptedAge'] = a_votes['AcceptedAge'] + 1

a_votes.loc[a_votes.AcceptedDate == pd.to_datetime('20160320'), 'AcceptedAge'] = -1

a_votes['RelAge'] = (pd.to_datetime(a_votes.VoteCreation,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(a_votes.AnsCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
a_votes['RelAge'] = a_votes['RelAge'] + 1

a_votes['Age'] = (pd.to_datetime(a_votes.VoteCreation,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(a_votes.QuestionCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
a_votes['Age'] = a_votes['Age'] + 1
a_votes.drop(a_votes.columns[[0, 1, 6, 8]], axis=1, inplace=True)
a_votes

,VoteType,AnsId,QuestionId,AnsWordCount,AcceptedAnsId,QScore,QVotes,AcceptedAge,RelAge,Age
0,2,4,1,190,56,0,0,6,0,0
1,2,4,1,190,56,0,0,6,0,0
2,2,5,1,37,56,0,0,6,0,0
3,3,4,1,190,56,0,0,6,0,0
4,2,6,2,51,10,0,0,1,0,0
5,2,10,2,145,10,0,0,1,0,0
6,2,13,2,89,10,0,0,1,0,0
7,2,8,2,46,10,0,0,1,0,0
8,2,13,2,89,10,0,0,1,0,0
9,2,10,2,145,10,0,0,1,0,0


In [34]:
a_comments = pd.DataFrame.from_csv('comment-ans.csv', index_col=None)
a_comments = pd.merge(a_comments, questions, how='inner', on=['QuestionId'],suffixes=['_vot', '_quest'])

a_comments['Age'] = (pd.to_datetime(a_comments.CommentCreation,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(a_comments.QuestionCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
a_comments['Age'] = a_comments['Age'] + 1
a_comments = a_comments[['AnsId','Age']]
a_comments.head(10)

,AnsId,Age
0,4,1
1,4,1
2,5,1
3,56,1
4,56,1
5,56,1
6,56,1
7,7030,951
8,7030,1285
9,15104,1436


## Each entry in the following tables represents the total votes in an answer in a given day (and associated attributes for the answer and question)

In [35]:
get_score = lambda df: sum(df.VoteType==2) - sum(df.VoteType==3)
get_votes = lambda df: sum(df.VoteType==2) + sum(df.VoteType==3)

predictors = ['QuestionId','AnsWordCount','AcceptedAnsId','AcceptedAge','QScore','QVotes','Score','Votes','Upvotes','Downvotes']
f = lambda df: pd.Series([df.QuestionId.iloc[0],df.AnsWordCount.iloc[0],df.AcceptedAnsId.iloc[0],df.AcceptedAge.iloc[0],
                          df.QScore.iloc[0],df.QVotes.iloc[0],get_score(df),get_votes(df),sum(df.VoteType==2),sum(df.VoteType==3)],index = predictors)
a_groups = a_votes.sort_values(by='Age').groupby(['AnsId','Age']).apply(f)
a_groups = a_groups.reset_index(level=[0,1],drop=False)
a_groups.head()

,AnsId,Age,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,Upvotes,Downvotes
0,4,0,1,190,56,6,0,0,1,3,2,1
1,4,1436,1,190,56,6,12,12,1,1,1,0
2,5,0,1,37,56,6,0,0,1,1,1,0
3,5,108,1,37,56,6,12,12,-1,1,0,1
4,6,0,2,51,10,1,0,0,1,1,1,0


In [36]:
cum_votes = lambda df: pd.Series(df['Votes'].cumsum() - df['Votes'],name='CumVotes')
cum_score = lambda df: pd.Series(df['Score'].cumsum() - df['Score'],name='CumScore')

f =lambda df: pd.concat([df[cname] for cname in df.columns.values.tolist()] + [cum_votes(df),cum_score(df)],axis=1)
a_groups_c = a_groups.sort_values(by='Age').groupby(['AnsId']).apply(f)
a_groups_c

,AnsId,Age,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,Upvotes,Downvotes,CumVotes,CumScore
0,4,0,1,190,56,6,0,0,1,3,2,1,0,0
9873,5380,0,5379,43,5380,0,0,0,1,1,1,0,0,0
9872,5378,0,5377,51,5378,0,0,0,2,2,2,0,0,0
9868,5375,0,5373,241,NaN,-1,0,0,1,1,1,0,0,0
2475,858,0,857,91,858,39,0,0,3,3,3,0,0,0
9857,5368,0,5367,235,5368,0,0,0,1,1,1,0,0,0
9852,5363,0,5360,389,5363,3,0,0,1,1,1,0,0,0
9851,5362,0,5360,44,5363,3,0,0,1,1,1,0,0,0
9849,5359,0,5357,136,5359,0,0,0,2,2,2,0,0,0
9844,5358,0,5350,428,NaN,-1,0,0,2,2,2,0,0,0


In [38]:
## SHOULD RETURN 0,3
tt = a_groups_c.groupby(['AnsId','Age']).Votes.count().reset_index(level=[0,1],drop=False)
tt[tt.Votes>1].shape

(0, 3)

In [39]:
a_groups_c['ReScore'] = a_groups_c['CumScore']/(a_groups_c['CumVotes']+1.0)
a_groups_c['QReScore'] = a_groups_c['QScore']/(a_groups_c['QVotes']+1.0)
#votes['TScore'] = np.power(votes['Score'],votes['age_vot'])
a_groups_c.head()

,AnsId,Age,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,Upvotes,Downvotes,CumVotes,CumScore,ReScore,QReScore
0,4,0,1,190,56,6,0,0,1,3,2,1,0,0,0,0
9873,5380,0,5379,43,5380,0,0,0,1,1,1,0,0,0,0,0
9872,5378,0,5377,51,5378,0,0,0,2,2,2,0,0,0,0,0
9868,5375,0,5373,241,NaN,-1,0,0,1,1,1,0,0,0,0,0
2475,858,0,857,91,858,39,0,0,3,3,3,0,0,0,0,0


### Careful in this cell!!! It introduces duplicate AnsId-pairs (ignore for now)

In [55]:
votes_com = pd.merge(a_groups_c, a_comments, how='left', on=['AnsId'],suffixes=['_vot', '_com'])
def shrink_comments(df):
    dd = {cname:[df[cname].iloc[0]] for cname in df.columns.values.tolist() if cname not in ['AnsId','Age_vot'] }
    dd.update({'Comments':[sum(df['Age_vot']>=df['Age_com'])]})
    return pd.DataFrame(dd)
votes_com_f = votes_com.groupby(['AnsId','Age_vot']).apply(shrink_comments).reset_index(level=[0,1],drop=False)

In [56]:
## SHOULD RETURN 0,3
tt = votes_com_f.groupby(['AnsId','Age_vot']).Votes.count().reset_index(level=[0,1],drop=False)
tt[tt.Votes>1].shape

(0, 3)

## First Time series cleaned (model for the number of votes)

# Check inconsistent entries

In [57]:
votes_com_f[(votes_com_f['AnsId'].isnull()) | (votes_com_f['Age_vot'].isnull()) | (votes_com_f['Votes'].isnull()) | (votes_com_f['Score'].isnull()) | (votes_com_f['Comments'].isnull())]

,AnsId,Age_vot,AcceptedAge,AcceptedAnsId,Age_com,AnsWordCount,Comments,CumScore,CumVotes,Downvotes,QReScore,QScore,QVotes,QuestionId,ReScore,Score,Upvotes,Votes


In [58]:
votes_com_f[votes_com_f.Age_vot < 0]

,AnsId,Age_vot,AcceptedAge,AcceptedAnsId,Age_com,AnsWordCount,Comments,CumScore,CumVotes,Downvotes,QReScore,QScore,QVotes,QuestionId,ReScore,Score,Upvotes,Votes


In [59]:
votes_com_f[votes_com_f.CumScore>votes_com_f.CumVotes]

,AnsId,Age_vot,AcceptedAge,AcceptedAnsId,Age_com,AnsWordCount,Comments,CumScore,CumVotes,Downvotes,QReScore,QScore,QVotes,QuestionId,ReScore,Score,Upvotes,Votes


In [60]:
#Accepted answers 687 and 990 didn't have acceptance dates registered?
votes_com_f[(votes_com_f['AcceptedAnsId'].notnull()) & (votes_com_f['AcceptedAge']<0)]

,AnsId,Age_vot,AcceptedAge,AcceptedAnsId,Age_com,AnsWordCount,Comments,CumScore,CumVotes,Downvotes,QReScore,QScore,QVotes,QuestionId,ReScore,Score,Upvotes,Votes
0,687,0,-1,687,1,31,0,0,0,0,0.000000,0,0,686,0.000000,5,5,5
0,687,4,-1,687,1,31,5,5,5,0,0.800000,4,4,686,0.833333,1,1,1
0,687,35,-1,687,1,31,5,6,6,0,0.800000,4,4,686,0.857143,1,1,1
0,687,435,-1,687,1,31,5,7,7,0,0.800000,4,4,686,0.875000,1,1,1
0,688,0,-1,687,1,26,0,0,0,0,0.000000,0,0,686,0.000000,4,4,4
0,689,0,-1,687,1,48,0,0,0,0,0.000000,0,0,686,0.000000,3,3,3
0,689,256,-1,687,1,48,4,3,3,0,0.800000,4,4,686,0.750000,1,1,1
0,690,0,-1,687,NaN,39,0,0,0,0,0.000000,0,0,686,0.000000,1,1,1
0,990,0,-1,990,1,62,0,0,0,0,0.000000,0,0,987,0.000000,1,1,1
0,990,121,-1,990,1,62,6,1,1,0,0.857143,6,6,987,0.500000,1,1,1


## Calculate Rank

In [61]:
from itertools import izip
def rank_ans(df,score_only,re_score):
    rk_name = "ReScore_rank" if re_score else "AnsRank"
    def rank_iter():
        cache = {}
        accepted = 0
        for row in df.itertuples():
            if re_score:
                cache[row.AnsId] = row.ReScore
            else :
                cache[row.AnsId] = row.Score
            # rank, nb_ans
            if (not score_only) and row.AcceptedAge>-1 and (row.AnsId == row.AcceptedAnsId) and row.Age_vot >=row.AcceptedAge:
                accepted = 1
                if row.AnsId in cache:
                    del cache[row.AnsId]
                yield (1,len(cache)+accepted,row.Index)
            else :
                rank = sorted(cache, key= lambda k:cache[k],reverse=True).index(row.AnsId) + 1 + accepted
                yield (rank,len(cache)+accepted,row.Index)
            
    ranks, ans_counts, indices = izip(*list(rank_iter())) #TODO: optimize for the future
    return [pd.Series(ranks,name=rk_name, index=indices), pd.Series(ans_counts,name="Ans_count", index=indices)]

predictors = ['AnsId','Comments','AnsWordCount','AcceptedAnsId','Age_vot','Score','Votes','Upvotes','Downvotes','CumScore','CumVotes','QScore','QVotes','ReScore','QReScore','AnsRank','ReScore_rank']
get_ranks = lambda df,score_only=False,re_score=False: pd.concat([df[cname] for cname in df.columns.values.tolist() if cname in predictors] + rank_ans(df,score_only,re_score),axis=1)
sort_age_score = lambda df: df.sort_values(by=['Age_vot','Score'],ascending=[True,False])

## Base data for model

In [62]:
votes_com_f = votes_com_f.groupby(['QuestionId']).apply(lambda df: get_ranks(sort_age_score(df)))
votes_com_f = votes_com_f.reset_index(level=[0],drop=False)
votes_com_f = votes_com_f.groupby(['QuestionId']).apply(lambda df: get_ranks(sort_age_score(df),score_only=True,re_score=True))
votes_com_f = votes_com_f.reset_index(level=[0],drop=False)

votes_com_f['Pbias'] = 1.0/votes_com_f['AnsRank']
votes_com_f['DRank'] = votes_com_f['AnsRank'] - votes_com_f['ReScore_rank']
votes_com_f.head(20)

,QuestionId,AnsId,Age_vot,AcceptedAnsId,AnsWordCount,Comments,CumScore,CumVotes,Downvotes,QReScore,...,QVotes,ReScore,Score,Upvotes,Votes,AnsRank,ReScore_rank,Ans_count,Pbias,DRank
0,1,56,0,56,201,0,0,0,0,0.000000,...,0,0.000000,2,2,2,1,1,1,1.000000,0
0,1,4,0,56,190,0,0,0,1,0.000000,...,0,0.000000,1,2,3,2,2,2,0.500000,0
0,1,5,0,56,37,0,0,0,0,0.000000,...,0,0.000000,1,1,1,3,3,3,0.333333,0
0,1,56,1,56,201,4,2,2,0,0.923077,...,12,0.666667,3,3,3,1,1,3,1.000000,0
0,1,210,1,56,40,0,0,0,0,0.923077,...,12,0.000000,1,1,1,2,2,4,0.500000,0
0,1,56,4,56,201,4,5,5,0,0.923077,...,12,0.833333,1,1,1,1,1,4,1.000000,0
0,1,210,4,56,40,0,1,1,0,0.923077,...,12,0.500000,1,1,1,2,2,4,0.500000,0
0,1,56,7,56,201,4,6,6,0,0.923077,...,12,0.857143,1,1,1,1,1,4,1.000000,0
0,1,897,108,56,68,0,0,0,0,0.923077,...,12,0.000000,1,1,1,2,3,5,0.500000,-1
0,1,5,108,56,37,1,1,1,1,0.923077,...,12,0.500000,-1,0,1,5,3,5,0.200000,2


In [63]:
sum_by_rank = lambda df: df.groupby('AnsRank').apply(lambda df: pd.Series([df.Votes.sum()],name='EPbias').to_frame()).unstack(level=-1).reset_index(level=0,drop=False)
get_ratio = lambda df: sum_by_rank(df).EPbias/(sum_by_rank(df).EPbias.sum())
ratio_per_rank = lambda df: pd.concat([sum_by_rank(df).AnsRank, get_ratio(df)],axis=1)
get_position_bias = lambda df: pd.merge(df,ratio_per_rank(df),how='inner',on=['AnsRank'])

votes = votes_com_f.groupby(['Ans_count']).apply(get_position_bias).reset_index(level=[0,1],drop=True)
votes.columns.values[-1] = "EPbias"
votes

,QuestionId,AnsId,Age_vot,AcceptedAnsId,AnsWordCount,Comments,CumScore,CumVotes,Downvotes,QReScore,...,ReScore,Score,Upvotes,Votes,AnsRank,ReScore_rank,Ans_count,Pbias,DRank,EPbias
0,1,56,0,56,201,0,0,0,0,0.000000,...,0.000000,2,2,2,1,1,1,1.000000,0,1.000000
1,2,10,0,10,145,0,0,0,0,0.000000,...,0.000000,3,3,3,1,1,1,1.000000,0,1.000000
2,3,9,0,109,25,0,0,0,0,0.000000,...,0.000000,4,4,4,1,1,1,1.000000,0,1.000000
3,7,16,0,2977,45,0,0,0,0,0.000000,...,0.000000,3,3,3,1,1,1,1.000000,0,1.000000
4,14,20,0,20,28,0,0,0,0,0.000000,...,0.000000,7,7,7,1,1,1,1.000000,0,1.000000
5,18,33,0,33,133,0,0,0,0,0.000000,...,0.000000,3,3,3,1,1,1,1.000000,0,1.000000
6,21,40,0,80,84,0,0,0,0,0.000000,...,0.000000,1,1,1,1,1,1,1.000000,0,1.000000
7,22,46,0,46,110,0,0,0,0,0.000000,...,0.000000,3,3,3,1,1,1,1.000000,0,1.000000
8,25,27,0,67,59,0,0,0,0,0.000000,...,0.000000,1,1,1,1,1,1,1.000000,0,1.000000
9,26,81,17,NaN,37,3,0,0,0,0.875000,...,0.000000,1,1,1,1,1,1,1.000000,0,1.000000


In [65]:
#GET NULL say yay!!!
tt = votes.groupby(['AnsId','Age_vot']).Votes.count().reset_index(level=[0,1],drop=False)
tt[tt.Votes>2].head()

,AnsId,Age_vot,Votes


In [67]:
votes.to_csv(path_or_buf='AnsVotes_TSeries.csv')

In [31]:
votes[['Age_vot','AnsWordCount','Comments','Ans_count']].describe()

,Age_vot,AnsWordCount,Comments,Ans_count
count,23328.000000,23328.000000,23328.000000,23328.000000
mean,212.595508,143.485039,2.033522,2.721622
std,380.532603,150.314528,2.851866,2.061537
min,0.000000,3.000000,0.000000,1.000000
25%,1.000000,51.000000,0.000000,1.000000
50%,7.000000,97.000000,1.000000,2.000000
75%,231.000000,178.000000,3.000000,3.000000
max,1504.000000,2142.000000,18.000000,12.000000


In [32]:
votes['Ans_count'].value_counts().head(15)

1     7132
2     6866
3     4033
4     2172
5     1169
6      535
10     484
7      283
8      273
9      240
11      73
12      68
Name: Ans_count, dtype: int64

In [25]:
#votes = votes[ votes.age_vot>=3]
votes = votes[votes.Age_vot<=7]
votes[['Score','Votes','CumScore','CumVotes','QScore','QVotes','ReScore','QReScore','Ans_count','DRank']].describe()

,Score,Votes,CumScore,CumVotes,QScore,QVotes,ReScore,QReScore,Ans_count,DRank
count,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000
mean,1.674785,1.787765,3.338020,3.505733,4.262030,4.482802,0.667576,0.523072,1.892125,0.069462
std,1.404004,1.326312,2.641695,2.575425,6.957248,7.212644,0.233654,0.380433,1.204971,0.589805
min,-3.000000,1.000000,-6.000000,1.000000,-7.000000,0.000000,-0.857143,-0.875000,1.000000,-4.000000
25%,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.500000,0.000000,1.000000,0.000000
50%,1.000000,1.000000,3.000000,3.000000,3.000000,3.000000,0.750000,0.666667,2.000000,0.000000
75%,2.000000,2.000000,5.000000,5.000000,6.000000,6.000000,0.833333,0.833333,2.000000,0.000000
max,12.000000,12.000000,18.000000,18.000000,58.000000,62.000000,0.947368,0.979592,9.000000,8.000000


In [26]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

# How does question quality relate to answer quality

In [124]:
print "Pearson r Score-position: ", votes['Score'].corr(votes['AnsRank'],method='pearson')

Pearson r Score-position:  -0.261224567951


In [107]:
formula = "np.log(Votes+1) ~ C(AnsRank,Diff) - 1"
model = sm.MixedLM.from_formula(formula, votes, re_formula="AnsRank", groups=votes["AnsId"])
result = model.fit()
print result.summary()

c:\Users\Joel\Anaconda2\lib\site-packages\statsmodels\base\model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
c:\Users\Joel\Anaconda2\lib\site-packages\statsmodels\regression\mixed_linear_model.py:1717: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   np.log(Votes + 1)
No. Observations:   12054     Method:               REML             
No. Groups:         4594      Scale:                0.0965           
Min. group size:    1         Likelihood:           -4144.7526       
Max. group size:    70        Converged:            Yes              
Mean group size:    2.6                                              
---------------------------------------------------------------------
                          Coef.  Std.Err.    z    P>|z| [0.025 0.975]
---------------------------------------------------------------------
C(AnsRank, Diff)[D.1]      0.892    0.045  19.975 0.000  0.805  0.980
C(AnsRank, Diff)[D.2]     -0.177    0.008 -21.518 0.000 -0.194 -0.161
C(AnsRank, Diff)[D.3]     -0.067    0.013  -5.304 0.000 -0.092 -0.042
C(AnsRank, Diff)[D.4]      0.020    0.023   0.871 0.384 -0.025  0.065
C(AnsRank, Diff)[D.5]     -0.023    

## OLS, Poisson, RobustLM